In [1]:
import pandas as pd

In [2]:
#what should we be matching on... 
#options are: 
#CARDINAL, DATE, EVENT, FAC, GPE, LANGUAGE, LAW, LOC, MONEY, NORP, 
#ORDINAL, ORG, PERCENT, PERSON, PRODUCT, QUANTITY, TIME, WORK_OF_ART
#taken from: https://spacy.io/models/en


In [3]:
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/mergedNewsData/mergedNER.tsv", sep="\t")

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (1,2,6,9,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.head(3)
print(df.columns)

Index(['Unnamed: 0', 'article_id', 'sourcedomain_id', 'date', 'title',
       'content', 'url_x', 'fips', 'source', 'description', 'onlinesince',
       'rank', 'state', 'city', 'lon', 'lat', 'county', 'total_population',
       'white_pct', 'black_pct', 'hispanic_pct', 'nonwhite_pct',
       'foreignborn_pct', 'female_pct', 'age29andunder_pct',
       'age65andolder_pct', 'median_hh_inc', 'clf_unemploy_pct', 'lesshs_pct',
       'lesscollege_pct', 'lesshs_whites_pct', 'lesscollege_whites_pct',
       'rural_pct', 'ruralurban_cc', 'predrt_0', 'predrt_12', 'predrt_3',
       'national', 'id', 'author', 'url', 'published', 'published_utc',
       'collection_utc', 'key', 'topics'],
      dtype='object')


In [ ]:
"""
NATIONAL NEWS DATE RANGE: Jan 1st, 2020 - December 31st, 2021
LOCAL NEWS DATE RANGE: April 1st, 2020 - December 31st, 2021
"""

In [10]:
#need to check the date ranges to make sure we are using the full dataset here
natDf = df[df["national"] == True]
print("NAT. MIN: " + str(min(natDf["date"])))
print("NAT. MAX: " + str(max(natDf["date"])))

locDf = df[df["national"] == False]
print("LOCAL MIN: " + str(min(locDf["date"])))
print("LOCAL MAX: " + str(max(locDf["date"])))



NAT. MIN: 2020-01-01
NAT. MAX: 2021-12-31
LOCAL MIN: 2020-04-01
LOCAL MAX: 2021-12-31


In [5]:
#IDEA: unpack the list into multiple columns, get those columns into multiple rows. Throw out rows that don't have types of interest. 
#then, do a groupby [entity, entity_type] which should give us the proper reversal we can map on. 
#if articles show up in the same row once then they need to be compared? 
df["topics"]

0          [('PERSON', 'Dan Shehan’s'), ('WORK_OF_ART', '...
1          [('DATE', 'This past Sunday, March 29th'), ('E...
2          [('PERSON', 'Kay Ivey'), ('DATE', 'Saturday'),...
3          [('GPE', 'Covington County'), ('DATE', 'Monday...
4          [('GPE', 'Alabama'), ('GPE', 'Covington County...
                                 ...                        
4769864    [('ORG', 'the National Archives'), ('PERSON', ...
4769865    [('PERSON', 'Boris Johnson'), ('EVENT', "New Y...
4769866    [('DATE', 'another eventful year'), ('DATE', '...
4769867    [('GPE', 'UK'), ('EVENT', 'the New Year Honour...
4769868    [('GPE', 'Northern Ireland'), ('DATE', '2021')...
Name: topics, Length: 4769869, dtype: object

In [137]:
#Alternate attempt to create reverse mapping 
testDf= df[["key", "topics"]].dropna(subset=["key", "topics"])
testDict = dict(zip(testDf["key"], testDf["topics"]))
#outDict = {val:[] for key, val in testDict.items()}

print("dict made")

outDict = {}
for outlet, entList in testDict.items(): 
    for ent in entList: 
        
        #add a bunch of things for entity types etc... 
        #can we make the pairwise comparison dictionary at the same time? 
        if ent not in outDict: 
            outDict[ent] = [outlet]
        else: 
            outDict[ent].append(outlet)
print(list(outDict.items())[:3])

dict made
[(('PERSON', 'Dan Shehan’s'), ['andalusiastarnews_Remember_When:_Shehan’s_Polio_story']), (('WORK_OF_ART', 'My Polio Story'), ['andalusiastarnews_Remember_When:_Shehan’s_Polio_story']), (('DATE', 'week'), ['andalusiastarnews_Remember_When:_Shehan’s_Polio_story', 'juneauempire_With_physical_contact_forbidden,_Juneau_finds_new_ways_to_socialize', 'theenterpriserecord_Have_you_ever_dealt_with_glitter?_|__Off_the_Record', 'boulderweekly_Facing_supply_disruptions,_Boulder_County_food_banks_prepare_for_the_future', 'boulderweekly_Facing_supply_disruptions,_Boulder_County_food_banks_prepare_for_the_future', 'boulderweekly_Facing_supply_disruptions,_Boulder_County_food_banks_prepare_for_the_future', 'craigdailypress_Over_a_Cup_of_Coffee:_Recipes_using_beef', "haxtunflemingherald_NJC_responds_to_conditions_under_Governor's_stay-at-home_order", 'waterburyrepublicanamerican_Calmer_(and_definitely_quieter)_WrestleMania_adapts_to_reality', 'waterburyrepublicanamerican_New_at_home_this_wee

In [115]:
#NOTE: this approach is way too slow, especially the part where we explode into multiple rows 
import ast
import re

entList = ["ORG","EVENT", "PERSON", "WORK_OF_ART", "PRODUCT"]

#test out idea for creating reverse mapping 
testDf = df[["key", "topics"]].head(80000).dropna(subset=["topics"])
testDf["topics"] = testDf["topics"].apply(list)

#time consuming part is these three print statements 
print("list applied")
#split lists of tuples into seperate rows (i.e. long format )
testDf = testDf.explode(["topics"])

print("exploded")

#split ent_type, entity pairs to columns 
testDf[["ent_type","entity"]] = pd.DataFrame(testDf["topics"].tolist(), index=testDf.index)

print("split")

#only care about certain entity types 
testDf = testDf[testDf["ent_type"].isin(entList)]

print("subset")

#cleaning function
def cleanEntities(inStr): 
    lower = inStr.lower()
    alphaNum = " ".join(re.sub("[^\w\s]", "", lower).split())
    return alphaNum

#show that our cleaning function works well 
print(cleanEntities("h3ll0!..   \nW0rlD&*  \t,,"))

#clean up dataframe a bit 
testDf = testDf.drop(columns=["topics"]).dropna(subset=["ent_type", "entity"]).reset_index(drop=True)

#clean up entities so they match eachother better 
testDf["entity"] = testDf["entity"].apply(cleanEntities)

print("cleaned")

reverseMapping = testDf.groupby(["ent_type", "entity"]).agg(list)
print("grouped")

list applied


KeyboardInterrupt: 

In [105]:
reverseMapping["len"] = reverseMapping["key"].map(len)
reverseMapping[reverseMapping["len"] >=2]

def create_mapping(inList): 
    create_mapping(inList)

key  \
ent_type    entity                                                                 
EVENT       allstar game       [dailyherald_Go_Figure:_Ed_Farmer's_numbers_te...   
            an olympic games   [dailybreeze_Coronavirus_delays_UCLA_pole_vaul...   
            celtics            [dailybreeze_Who_else_is_joining_Kobe_Bryant_i...   
            championships      [westhawaiitoday_Virus_forces_Wimbledon_cancel...   
            division ii        [contracostatimes_Kobe_Bryant,_Tim_Duncan_head...   
...                                                                          ...   
WORK_OF_ART today              [dailyreviewatlas_'I_just_don't_understand':_A...   
            top class actions  [dailyreviewatlas_Coronavirus_goes_to_court:_A...   
            top gun maverick   [waterburyrepublicanamerican_‘Top_Gun_Maverick...   
            we are big         [dailyreviewatlas_Coronavirus_goes_to_court:_A...   
            world on fire      [waterburyrepublicanamerican_Commentary:_Two_b...   

                               len  
ent_type    entity                  
EVENT       allstar game         2  
            an olympic games     3  
            celtics              3  
            championships        2  
            division ii          8  
...                            ...  
WORK_OF_ART today                4  
            top class actions    4  
            top gun maverick     2  
            we are big           4  
            world on fire        4  

[2361 rows x 2 columns]

In [63]:
testDf = df[["key", "topics"]].head(1000).dropna()
testDf["topics"] = testDf["topics"].apply(ast.literal_eval)


ValueError: malformed node or string: [('PERSON', 'Dan Shehan’s'), ('WORK_OF_ART', 'My Polio Story'), ('DATE', 'week'), ('DATE', 'the 1950s'), ('DATE', 'today'), ('ORG', 'Coronavirus'), ('PERSON', 'Morgan Moore'), ('PERSON', 'Kenneth Moore Hannon'), ('GPE', 'Andalusia'), ('GPE', 'Mobile'), ('ORG', 'the Crippled Children’s Clinic'), ('FAC', 'the Scherf Memorial Building'), ('FAC', 'Opp Avenue'), ('ORG', 'the Red Cross'), ('GPE', 'Andalusia'), ('PERSON', 'Hannon'), ('ORG', 'Medicaid'), ('PERSON', 'Lawrence Bowness'), ('GPE', 'Chicago'), ('GPE', 'Chicago'), ('DATE', 'ten year old'), ('TIME', 'one morning'), ('GPE', 'Pensacola'), ('PERSON', 'Robert Earl Vickery'), ('GPE', 'Birmingham'), ('GPE', 'Pensacola'), ('DATE', 'the middle of the'), ('ORG', 'Navy'), ('GPE', 'Birmingham'), ('CARDINAL', 'two'), ('PERSON', 'Norman'), ('PERSON', 'Henry Everage'), ('ORG', 'Jefferson Hillman Hospital'), ('DATE', 'the next six months'), ('ORDINAL', '11th'), ('DATE', 'the spring of 1954'), ('GPE', 'Gonzalez'

,0,1
0,PERSON,Dan Shehan’s
1,WORK_OF_ART,My Polio Story
2,DATE,week
3,DATE,the 1950s
4,DATE,today
...,...,...
43170,QUANTITY,100-mile
43171,PERSON,Ramon Padilla
43172,PERSON,Mitchell Thorson
43173,ORG,USA TODAY Network
